# live test 

encoder : 24 *7 , decoder : 24  
실제 일자별 예측을 시각화 및 gif 로 표현하는 노트북  

In [1]:
# package load 
import os
import json
import copy
import torch
import warnings
import numpy as np
import pandas as pd
import lightning.pytorch as pl
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.offline as pyo


from PIL import Image
from pathlib import Path
from natsort import natsorted
from tqdm import tqdm_notebook
from matplotlib import gridspec
from collections import defaultdict
from sklearn.metrics import mean_squared_error , confusion_matrix
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

from IPython.core.display import HTML
from plotly.subplots import make_subplots

from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.metrics.base_metrics import MultiHorizonMetric

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

In [2]:
from PIL import Image
import os

def images_to_gif(image_files, output_file):
    images = [Image.open(image_file) for image_file in image_files]
    images[0].save(output_file, save_all=True, append_images=images[1:], optimize=False, duration=700, loop=0)


In [3]:
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import torch
from pytorch_forecasting.metrics.base_metrics import MultiHorizonMetric

class CustomLoss(MultiHorizonMetric):
    """
    Quantile loss, i.e. a quantile of ``q=0.5`` will give half of the mean absolute error as it is calculated as

    Defined as ``max(q * (y-y_pred), (1-q) * (y_pred-y))``
    """

    def __init__(
        self,
        threshold: float = 0.5,
        quantiles: List[float] = [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98],
        **kwargs,
    ):
        """
        Quantile loss

        Args:
            quantiles: quantiles for metric
        """
        self.threshold = threshold
        super().__init__(quantiles=quantiles, **kwargs)

    def loss(self, y_pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        # calculate quantile loss
        losses = []
        for i, q in enumerate(self.quantiles):
            errors = target - y_pred[..., i]
            losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(-1) * (self.calculate_f1_loss(target , - y_pred[..., i])))
        losses = 2 * torch.cat(losses, dim=2)

        return losses
    
    def calculate_f1_loss(self, y_true: torch.Tensor, y_pred: torch.Tensor) -> torch.Tensor:
        y_true_binary = (y_true > self.threshold).float()
        #y_pred = self.to_prediction(y_pred)
        y_pred_binary = (y_pred > self.threshold).float()

        tp = torch.sum(y_true_binary * y_pred_binary, dim=0)
        fp = torch.sum((1 - y_true_binary) * y_pred_binary, dim=0)
        fn = torch.sum(y_true_binary * (1 - y_pred_binary), dim=0)

        epsilon = 1e-7  # to prevent division by zero
        precision = tp / (tp + fp + epsilon)
        recall = tp / (tp + fn + epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + epsilon)
        return 1 - f1.mean()  # return 1 minus mean f1 score to make it a loss (lower is better)


    def to_prediction(self, y_pred: torch.Tensor) -> torch.Tensor:
        """
        Convert network prediction into a point prediction.

        Args:
            y_pred: prediction output of network

        Returns:
            torch.Tensor: point prediction
        """
        if y_pred.ndim == 3:
            idx = self.quantiles.index(0.5)
            y_pred = y_pred[..., idx]
        return y_pred

    def to_quantiles(self, y_pred: torch.Tensor) -> torch.Tensor:
        """
        Convert network prediction into a quantile prediction.

        Args:
            y_pred: prediction output of network

        Returns:
            torch.Tensor: prediction quantiles
        """
        return y_pred



In [4]:
transform_type = 'ControlGroup'

# transform_type = lossGroup 으로 해서 코드 실행


quantiles = [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98]
alpha_start = 0.7

filter5 = np.array([2, 1, 0]) / 3
filter = np.exp(-np.arange(len(filter5)))

def triangle_conv(time_series):
    global filter
    size = len(filter)
    shift = size // 2
    conv_result = np.convolve(time_series, filter, mode='same')
    conv_result = np.roll(conv_result, shift)
    return conv_result

def inv_conv(time_series):
    return time_series


transformation_dict = {
    'forward': triangle_conv,
    'reverse': inv_conv, 
}


In [5]:
train_data = pd.read_csv('../../DataProcessing/train_data/Long Beach_5.csv')
train_data = train_data[-744*15:]
x_range = train_data.OccTime.unique()

In [6]:
directory_path = './sparity_test_configs'
trail_num = 0
configs = natsorted(os.listdir(directory_path))
filtered_list = [file for file in configs if not file.startswith('.ipy')]
file_path = os.path.join(directory_path , filtered_list[trail_num])
with open(file_path, 'r') as json_file:
    config_p = json.load(json_file)
    

In [7]:
config_p['conv_none_model_ckpt']

'lightning_logs/ControlGroup/lightning_logs/version_12/checkpoints/epoch=9-step=22170.ckpt'

In [8]:
#val_datas = natsorted(os.listdir('../../DataProcessing/train_data/Long Beach_5_val'))
val_datas = natsorted(os.listdir('../../DataProcessing/train_data/Long Beach_5_val'), reverse=True)
    
    
max_prediction_length = 24
max_encoder_length = 24*7
data = pd.read_csv('../../DataProcessing/train_data/Long Beach_5_split.csv')
training_cutoff = data["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Accient",
    group_ids=["GeoHash"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=[],
    time_varying_known_reals=data.columns[4: -2].tolist(),
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[],
    target_normalizer=GroupNormalizer(
        groups=["GeoHash"], transformation=None
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 256  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
#val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)


### conv_none_live_test 

In [9]:
load_tft = TemporalFusionTransformer.load_from_checkpoint(config_p['conv_none_model_ckpt'])
predictions_list = []
real_list = []
figs_dict = defaultdict(lambda: make_subplots(specs=[[{"secondary_y": True}]]))  # dict of figures, one per GeoHash
steps_dict = defaultdict(list)  # dict of steps, one list per GeoHash

for val_idx , val_data_list in enumerate(tqdm_notebook(val_datas , leave=False , desc = 'date processing')):
    val_data_path = os.path.join('../../DataProcessing/train_data/Long Beach_5_val' , val_data_list)
    val_data = pd.read_csv(val_data_path)
    validation = TimeSeriesDataSet.from_dataset(training, val_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

    predictions = load_tft.predict(val_dataloader, mode="raw" , return_x=True , return_index=True ,  trainer_kwargs=dict(accelerator="cpu"))
    raw_predictions = predictions[0]
    x = predictions[1]
    idx_df = predictions[2]

    for idx in range(raw_predictions['prediction'].shape[0]):
        group = idx_df.loc[idx, 'GeoHash']
        fig_plolty = figs_dict[group]  # get or create figure for this GeoHash
        steps = steps_dict[group]  # get or create steps list for this GeoHash

        y_pred = raw_predictions['prediction'][idx].unsqueeze(0)  # (sequence_length, num_quantiles) -> (1, sequence_length, num_quantiles)
        y_true = x['decoder_target'][idx].unsqueeze(0)  # (sequence_length,) -> (1, sequence_length, 1)

        loss_fn = QuantileLoss()
        loss = loss_fn(y_pred, y_true)

        for quantile_idx in range(len(quantiles)):
            quantile = quantiles[quantile_idx]  # Access the quantile value from the list
            alpha_color = alpha_start - quantile_idx / len(quantiles) * 0.7  # Calculate alpha value based on quantile index
            fillcolor = f'rgba(0, 0, 255, {alpha_color})'  # Set the fill color with adjusted alpha value

            fig_plolty.add_trace(go.Scatter(
                x=np.arange(0,24),
                y=raw_predictions['prediction'][idx, :, quantile_idx],
                mode='lines',
                name=f'Quantile {quantile_idx}',
                line=dict(color='blue'),
                fill='tozeroy',  # Fill the area below the line
                fillcolor='rgba(0, 0, 255, 0.7)',  # Set the fill color with adjusted alpha value
            ), row=1, col=1)

        fig_plolty.add_trace(go.Scatter(
            x=np.arange(0,24),
            y=x['encoder_target'][idx],
            mode='lines',
            name='Encoder Real',
            line=dict(color='black')
        ), row=1, col=1)

        actual_trace = go.Scatter(
            x=np.arange(-168+24,24),
            y=x['encoder_target'][idx],
            mode='lines',
            name='Encoder Real',
            line=dict(color='black')
        )
        fig_plolty.add_trace(actual_trace)

        attention_data = raw_predictions['encoder_attention'][idx].mean(dim=(0, 1))

        fig_plolty.add_trace(go.Scatter(
            x=np.arange(-168+24, 24),
            y=attention_data,
            mode='lines',
            name='Encoder Attention',
            line=dict(color='rgba(128, 128, 128, 0.6)')  # Set the color to gray with opacity 0.6
        ), secondary_y=True, row=1, col=1)

        step = dict(
            method="update",
            args=[
                {"visible": [j//10 == val_idx for j in range(len(val_datas)*11)]},
                {"title": f"Plot Prediction (Loss: {loss.item():.8f}) {group} date {val_idx + 1}"},
            ],
        )
        steps.append(step)

for group in figs_dict.keys():
    fig_plolty = figs_dict[group]
    steps = steps_dict[group]
    fig_plolty.update_layout(
        sliders=[dict(
            steps=steps,
        )]
    )

    filename = f"./{transform_type}_result/live_test/conv_none/{group}.html"  # modify this to specify your preferred filename and path
    dir_path = os.path.dirname(filename)  # Get the directory path
    os.makedirs(dir_path, exist_ok=True)  # Create the directory if it does not exist
    pyo.plot(fig_plolty, filename=filename)

date processing:   0%|          | 0/31 [00:00<?, ?it/s]

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
I

### concateloss result 


In [10]:
load_tft = TemporalFusionTransformer.load_from_checkpoint(config_p['concateloss_model_ckpt'])
predictions_list = []
real_list = []
figs_dict = defaultdict(lambda: make_subplots(specs=[[{"secondary_y": True}]]))  # dict of figures, one per GeoHash
steps_dict = defaultdict(list)  # dict of steps, one list per GeoHash

for val_idx , val_data_list in enumerate(tqdm_notebook(val_datas , leave=False , desc = 'date processing')):
    val_data_path = os.path.join('../../DataProcessing/train_data/Long Beach_5_val' , val_data_list)
    val_data = pd.read_csv(val_data_path)
    validation = TimeSeriesDataSet.from_dataset(training, val_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

    predictions = load_tft.predict(val_dataloader, mode="raw" , return_x=True , return_index=True ,  trainer_kwargs=dict(accelerator="cpu"))
    raw_predictions = predictions[0]
    x = predictions[1]
    idx_df = predictions[2]

    for idx in range(raw_predictions['prediction'].shape[0]):
        group = idx_df.loc[idx, 'GeoHash']
        fig_plolty = figs_dict[group]  # get or create figure for this GeoHash
        steps = steps_dict[group]  # get or create steps list for this GeoHash

        y_pred = raw_predictions['prediction'][idx].unsqueeze(0)  # (sequence_length, num_quantiles) -> (1, sequence_length, num_quantiles)
        y_true = x['decoder_target'][idx].unsqueeze(0)  # (sequence_length,) -> (1, sequence_length, 1)

        loss_fn = QuantileLoss()
        loss = loss_fn(y_pred, y_true)

        for quantile_idx in range(len(quantiles)):
            quantile = quantiles[quantile_idx]  # Access the quantile value from the list
            alpha_color = alpha_start - quantile_idx / len(quantiles) * 0.7  # Calculate alpha value based on quantile index
            fillcolor = f'rgba(0, 0, 255, {alpha_color})'  # Set the fill color with adjusted alpha value

            fig_plolty.add_trace(go.Scatter(
                x=np.arange(0,24),
                y=raw_predictions['prediction'][idx, :, quantile_idx],
                mode='lines',
                name=f'Quantile {quantile_idx}',
                line=dict(color='blue'),
                fill='tozeroy',  # Fill the area below the line
                fillcolor='rgba(0, 0, 255, 0.7)',  # Set the fill color with adjusted alpha value
            ), row=1, col=1)

        fig_plolty.add_trace(go.Scatter(
            x=np.arange(0,24),
            y=x['encoder_target'][idx],
            mode='lines',
            name='Encoder Real',
            line=dict(color='black')
        ), row=1, col=1)

        actual_trace = go.Scatter(
            x=np.arange(-168+24,24),
            y=x['encoder_target'][idx],
            mode='lines',
            name='Encoder Real',
            line=dict(color='black')
        )
        fig_plolty.add_trace(actual_trace)

        attention_data = raw_predictions['encoder_attention'][idx].mean(dim=(0, 1))

        fig_plolty.add_trace(go.Scatter(
            x=np.arange(-168+24, 24),
            y=attention_data,
            mode='lines',
            name='Encoder Attention',
            line=dict(color='rgba(128, 128, 128, 0.6)')  # Set the color to gray with opacity 0.6
        ), secondary_y=True, row=1, col=1)

        step = dict(
            method="update",
            args=[
                {"visible": [j//10 == val_idx for j in range(len(val_datas)*11)]},
                {"title": f"Plot Prediction (Loss: {loss.item():.8f}) {group} date {val_idx + 1}"},
            ],
        )
        steps.append(step)

for group in figs_dict.keys():
    fig_plolty = figs_dict[group]
    steps = steps_dict[group]
    fig_plolty.update_layout(
        sliders=[dict(
            steps=steps,
        )]
    )

    filename = f"./{transform_type}_result/live_test/concateloss/{group}.html"  # modify this to specify your preferred filename and path
    dir_path = os.path.dirname(filename)  # Get the directory path
    os.makedirs(dir_path, exist_ok=True)  # Create the directory if it does not exist
    pyo.plot(fig_plolty, filename=filename)

date processing:   0%|          | 0/31 [00:00<?, ?it/s]

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
I

### nei result 

In [11]:
import geohash
import math

def l2_distance(lat1, lon1, lat2, lon2):
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    d_lat = lat2_rad - lat1_rad
    d_lon = lon2_rad - lon1_rad
    
    distance = math.sqrt(d_lat**2 + d_lon**2)
    return distance


def get_nei_geohash_max(geohashes):
    nei_ghs = geohash.neighbors(geohashes)
    lat, lon = geohash.decode(geohashes)

    decoded_list = [geohash.decode(gh) for gh in nei_ghs]
    l2_distances = [l2_distance(lat, lon, lat_decoded, lon_decoded) for lat_decoded, lon_decoded in decoded_list]
    max_distance = max(l2_distances)

    # Get the indices of the top 4 geohashes with the largest L2 distances
    top_indices = sorted(range(len(l2_distances)), key=lambda i: l2_distances[i], reverse=True)[:4]
    geohashes_with_max_distance = [nei_ghs[i] for i in top_indices]

    return geohashes_with_max_distance

def get_nei_geohash_min(geohashes):
    nei_ghs = geohash.neighbors(geohashes)
    lat, lon = geohash.decode(geohashes)

    decoded_list = [geohash.decode(gh) for gh in nei_ghs]
    l2_distances = [l2_distance(lat, lon, lat_decoded, lon_decoded) for lat_decoded, lon_decoded in decoded_list]
    min_distance = min(l2_distances)

    # Get the indices of the top 4 geohashes with the smallest L2 distances
    top_indices = sorted(range(len(l2_distances)), key=lambda i: l2_distances[i])[:4]
    geohashes_with_min_distance = [nei_ghs[i] for i in top_indices]

    return geohashes_with_min_distance


In [12]:
data = pd.read_csv('../../DataProcessing/train_data/Long Beach_5_split.csv')

for ghs in data['GeoHash'].unique():
    gh_data = data[data['GeoHash'] == ghs]

    # Get max data for each neighboring geohash
    max_data = []
    for inner_nei in get_nei_geohash_max(ghs):
        inner_nei_data = data[data['GeoHash'] == inner_nei]['Accient'].to_list()
        if inner_nei_data:  # Check if the list is not empty
            max_data.append([val * 1.0 for val in inner_nei_data])

    # Get min data for each neighboring geohash
    min_data = []
    for inner_nei in get_nei_geohash_min(ghs):
        inner_nei_data = data[data['GeoHash'] == inner_nei]['Accient'].to_list()
        if inner_nei_data:  # Check if the list is not empty
            min_data.append([val * 1.0 for val in inner_nei_data])

    # Calculate the sum of max_data and min_data
    sum_max_data = np.sum(max_data, axis=0) if max_data else []
    sum_min_data = np.sum(min_data, axis=0) if min_data else []
    gh_data['nei'] = gh_data['Accient'] + sum_max_data + sum_min_data
    data.loc[gh_data.index , "nei"] = gh_data['nei'] + sum_max_data + sum_min_data
data['nei'].fillna(0, inplace=True)
training_cutoff = data["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Accient",
    group_ids=["GeoHash"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=[],
    time_varying_known_reals=[],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=data.columns[4: -3].tolist() + ['nei'],
    target_normalizer=GroupNormalizer(
        groups=["GeoHash"], transformation=None
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size=256
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)


In [13]:
load_tft = TemporalFusionTransformer.load_from_checkpoint(config_p['nei_model_ckpt'])
predictions_list = []
real_list = []
figs_dict = defaultdict(lambda: make_subplots(specs=[[{"secondary_y": True}]]))  # dict of figures, one per GeoHash
steps_dict = defaultdict(list)  # dict of steps, one list per GeoHash

for val_idx , val_data_list in enumerate(tqdm_notebook(val_datas , leave=False , desc = 'date processing')):
    val_data_path = os.path.join('../../DataProcessing/train_data/Long Beach_5_val' , val_data_list)
    val_data = pd.read_csv(val_data_path)
    
    for ghs in val_data['GeoHash'].unique():
        gh_data = val_data[val_data['GeoHash'] == ghs]

        # Get max data for each neighboring geohash
        max_data = []
        for inner_nei in get_nei_geohash_max(ghs):
            inner_nei_data = val_data[val_data['GeoHash'] == inner_nei]['Accient'].to_list()
            if inner_nei_data:  # Check if the list is not empty
                max_data.append([val * 1.0 for val in inner_nei_data])

        # Get min data for each neighboring geohash
        min_data = []
        for inner_nei in get_nei_geohash_min(ghs):
            inner_nei_data = val_data[val_data['GeoHash'] == inner_nei]['Accient'].to_list()
            if inner_nei_data:  # Check if the list is not empty
                min_data.append([val * 1.0 for val in inner_nei_data])

        # Calculate the sum of max_data and min_data
        sum_max_data = np.sum(max_data, axis=0) if max_data else []
        sum_min_data = np.sum(min_data, axis=0) if min_data else []
        gh_data['nei'] = gh_data['Accient'] + sum_max_data + sum_min_data
        val_data.loc[gh_data.index , "nei"] = gh_data['nei'] + sum_max_data + sum_min_data
    val_data['nei'].fillna(0, inplace=True)

    
    training_cutoff = data["time_idx"].max() - max_prediction_length

    validation = TimeSeriesDataSet.from_dataset(training, val_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

    predictions = load_tft.predict(val_dataloader, mode="raw" , return_x=True , return_index=True ,  trainer_kwargs=dict(accelerator="cpu"))
    raw_predictions = predictions[0]
    x = predictions[1]
    idx_df = predictions[2]

    for idx in range(raw_predictions['prediction'].shape[0]):
        group = idx_df.loc[idx, 'GeoHash']
        fig_plolty = figs_dict[group]  # get or create figure for this GeoHash
        steps = steps_dict[group]  # get or create steps list for this GeoHash

        y_pred = raw_predictions['prediction'][idx].unsqueeze(0)  # (sequence_length, num_quantiles) -> (1, sequence_length, num_quantiles)
        y_true = x['decoder_target'][idx].unsqueeze(0)  # (sequence_length,) -> (1, sequence_length, 1)

        loss_fn = QuantileLoss()
        loss = loss_fn(y_pred, y_true)

        for quantile_idx in range(len(quantiles)):
            quantile = quantiles[quantile_idx]  # Access the quantile value from the list
            alpha_color = alpha_start - quantile_idx / len(quantiles) * 0.7  # Calculate alpha value based on quantile index
            fillcolor = f'rgba(0, 0, 255, {alpha_color})'  # Set the fill color with adjusted alpha value

            fig_plolty.add_trace(go.Scatter(
                x=np.arange(0,24),
                y=raw_predictions['prediction'][idx, :, quantile_idx],
                mode='lines',
                name=f'Quantile {quantile_idx}',
                line=dict(color='blue'),
                fill='tozeroy',  # Fill the area below the line
                fillcolor='rgba(0, 0, 255, 0.7)',  # Set the fill color with adjusted alpha value
            ), row=1, col=1)

        fig_plolty.add_trace(go.Scatter(
            x=np.arange(0,24),
            y=x['encoder_target'][idx],
            mode='lines',
            name='Encoder Real',
            line=dict(color='black')
        ), row=1, col=1)

        actual_trace = go.Scatter(
            x=np.arange(-168+24,24),
            y=x['encoder_target'][idx],
            mode='lines',
            name='Encoder Real',
            line=dict(color='black')
        )
        fig_plolty.add_trace(actual_trace)

        attention_data = raw_predictions['encoder_attention'][idx].mean(dim=(0, 1))

        fig_plolty.add_trace(go.Scatter(
            x=np.arange(-168+24, 24),
            y=attention_data,
            mode='lines',
            name='Encoder Attention',
            line=dict(color='rgba(128, 128, 128, 0.6)')  # Set the color to gray with opacity 0.6
        ), secondary_y=True, row=1, col=1)

        step = dict(
            method="update",
            args=[
                {"visible": [j//10 == val_idx for j in range(len(val_datas)*11)]},
                {"title": f"Plot Prediction (Loss: {loss.item():.8f}) {group} date {val_idx + 1}"},
            ],
        )
        steps.append(step)

for group in figs_dict.keys():
    fig_plolty = figs_dict[group]
    steps = steps_dict[group]
    fig_plolty.update_layout(
        sliders=[dict(
            steps=steps,
        )]
    )

    filename = f"./{transform_type}_result/live_test/nei/{group}.html"  # modify this to specify your preferred filename and path
    dir_path = os.path.dirname(filename)  # Get the directory path
    os.makedirs(dir_path, exist_ok=True)  # Create the directory if it does not exist
    pyo.plot(fig_plolty, filename=filename)

date processing:   0%|          | 0/31 [00:00<?, ?it/s]

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
I